In [8]:
import os
import pandas
import pickle
import random

from tqdm import tqdm

In [16]:
trainFile = 'recurrent/train.pickle'
testFile = 'recurrent/test.pickle'

validation_split = 0.2
samples_per_session = 250

framenumName = 'Unnamed: 0'
features = ['paw1LH_top', 'paw2LF_top', 'paw3RF_top', 'paw4RH_top', 'tailBase_top', 'tailMid_top']

In [17]:
cols = list(range(19))

In [20]:
train = {}
test = {}

for session in tqdm(os.listdir('recurrent/sessions')):
    if session[0] == '.':
        continue
    train[session] = []
    test[session] = []
    tmp = pandas.read_csv(os.path.join('recurrent', 'sessions', session, 'trackedFeaturesRaw.csv'), usecols=cols)
    tmp = tmp.to_dict('records')
    samples = random.sample(tmp, len(tmp))
    i = -1
    while len(test[session]) < int(validation_split*samples_per_session):
        i += 1
        if any(samples[i][feature+'.2'] < 0.99 for feature in features):
            continue
        test[session].append({**{'frameNum': int(samples[i][framenumName])}, **{feature: (samples[i][feature], samples[i][feature+'.1']) for feature in features}})
    while len(train[session]) < int((1-validation_split)*samples_per_session):
        i += 1
        if any(samples[i][feature+'.2'] < 0.99 for feature in features):
            continue
        train[session].append({**{'frameNum': int(samples[i][framenumName])}, **{feature: (samples[i][feature], samples[i][feature+'.1']) for feature in features}})
                



  0%|          | 0/17 [00:00<?, ?it/s]

 12%|█▏        | 2/17 [00:07<00:53,  3.57s/it]

 18%|█▊        | 3/17 [00:12<00:59,  4.28s/it]

 24%|██▎       | 4/17 [00:18<01:01,  4.72s/it]

 29%|██▉       | 5/17 [00:26<01:03,  5.27s/it]

 35%|███▌      | 6/17 [00:33<01:00,  5.51s/it]

 41%|████      | 7/17 [00:39<00:55,  5.58s/it]

 47%|████▋     | 8/17 [00:43<00:48,  5.43s/it]

 53%|█████▎    | 9/17 [00:48<00:43,  5.39s/it]

 59%|█████▉    | 10/17 [00:53<00:37,  5.32s/it]

 65%|██████▍   | 11/17 [01:03<00:34,  5.80s/it]

 71%|███████   | 12/17 [01:09<00:28,  5.78s/it]

 76%|███████▋  | 13/17 [01:14<00:22,  5.70s/it]

 82%|████████▏ | 14/17 [01:18<00:16,  5.60s/it]

 88%|████████▊ | 15/17 [01:22<00:11,  5.50s/it]

 94%|█████████▍| 16/17 [01:26<00:05,  5.43s/it]

100%|██████████| 17/17 [01:31<00:00,  5.36s/it]



In [23]:
for session in test:
    for frame in test[session]:
        frame['frameNum'] = int(frame['frameNum'])

for session in train:
    for frame in train[session]:
        frame['frameNum'] = int(frame['frameNum'])

In [26]:
with open(trainFile, 'wb') as f:
    pickle.dump(train, f)

with open(testFile, 'wb') as f:
    pickle.dump(test, f)